In [5]:
 !wget https://www.dropbox.com/s/xq4vosn9xyn1dy1/grouped_data.pickle?dl=0 -O grouped_data.pickle

--2021-05-01 21:39:16--  https://www.dropbox.com/s/xq4vosn9xyn1dy1/grouped_data.pickle?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6027:18::a27d:4812
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/xq4vosn9xyn1dy1/grouped_data.pickle [following]
--2021-05-01 21:39:16--  https://www.dropbox.com/s/raw/xq4vosn9xyn1dy1/grouped_data.pickle
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc2460620f0db9b5875782387d4d.dl.dropboxusercontent.com/cd/0/inline/BNo5ng8g7hof_EvNopL-_2qF9ljoTHGQTN4loE974vAaGmj9vo5FGpKRI5Xo9O0EjLCwT642-7RLStjFujsNqV0iSa0_l8FCNx-3T0knfY_wlPjl2KqSG5SZA4HjH8XenSMo3qjNjoFoGC9s44kdb3NJ/file# [following]
--2021-05-01 21:39:16--  https://uc2460620f0db9b5875782387d4d.dl.dropboxusercontent.com/cd/0/inline/BNo5ng8g7hof_EvNopL-_2qF9ljoTHGQTN4loE974vAaGmj9vo5FGpKRI5Xo9

In [1]:
import torch
import torch.nn as nn
!pip install transformers
from transformers import BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
from transformers import BertForSequenceClassification, BertTokenizerFast, Trainer, TrainingArguments
import numpy as np
import torch.optim as optim
import pandas as pd
import torch
import copy
import pickle
import time
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm

     |████████████████████████████████| 2.1MB 12.9MB/s 
     |████████████████████████████████| 901kB 54.5MB/s 
     |████████████████████████████████| 3.3MB 50.7MB/s 


In [2]:
# Check device availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
print("You are using device: %s" % device)

You are using device: cuda


In [6]:
class BERT():
    def __init__(self, path='bert-base-uncased', cuda=False):
        self.tokenizer = BertTokenizer.from_pretrained(path)
        self.model = BertModel.from_pretrained(path)

        self.cuda = cuda

        self.load_model()
        pass

    def convert(self, words_list, method='average'):
        tokens = self.tokenizer.encode(' '.join(words_list))
        input_ids = torch.tensor(tokens).unsqueeze(0)

        if self.cuda:
            input_ids.cuda()
        outputs = self.model(input_ids)
        last_hidden_states = outputs[0]
        return last_hidden_states.squeeze(0).mean(axis=0).cpu().detach().numpy()
    
    def load_model(self):
        pass

In [7]:
with open('grouped_data.pickle', 'rb') as f:
    data = pickle.load(f)

In [8]:
bert = BERT(path='bert-base-uncased', cuda=torch.cuda.is_available())
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased', use_cache=True)

In [15]:
min = 0
max = 5000

In [ ]:
texts = []
labels = []
for key in tqdm(list(data.keys())[min:max]):
    post = data[key][0]
    comments = data[key][1]
    title = post['title']
    try:
        text = post['selftext']
    except:
        text = ""
    
    full_post = title + " " + text
    for comment in comments:
        if comment["body"] != "[deleted]":
            post_encodings = bert.convert(tokenizer(full_post))
            comment_encodings = bert.convert(tokenizer(comment["body"]))
            texts.append(np.array([post_encodings, comment_encodings]))
            labels.append(comment["score"])


Token indices sequence length is longer than the specified maximum sequence length for this model (632 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
pickle.dump([texts, labels], open('%d-%d.p' % (min,max), 'wb'))